In [1]:
import nltk
nltk.download('punkt')
nltk.download('wordnet')
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
import json
import pickle

import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
from keras.optimizers import SGD
import random




[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\LIMOS\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\LIMOS\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
Using TensorFlow backend.


In [2]:
words=[]
classes = []
documents = []
ignore_words = ['?', '!']
data_file = open('intents.json').read()
intents = json.loads(data_file)


for intent in intents['intents']:
    for pattern in intent['patterns']:
        w = nltk.word_tokenize(pattern)
        words.extend(w)
        documents.append((w, intent['tag']))
        if intent['tag'] not in classes:
            classes.append(intent['tag'])

words = [lemmatizer.lemmatize(w.lower()) for w in words if w not in ignore_words]
words = sorted(list(set(words)))

classes = sorted(list(set(classes)))

print (len(documents), "documents",documents)

print (len(classes), "classes", classes)

print (len(words), "unique lemmatized words", words)




31 documents [(['Hi', 'there'], 'greeting'), (['How', 'are', 'you'], 'greeting'), (['Is', 'anyone', 'there', '?'], 'greeting'), (['Hey'], 'greeting'), (['Hola'], 'greeting'), (['Hello'], 'greeting'), (['I', "'m", 'Good'], 'good'), (['Great'], 'good'), (['I', "'m", 'Sad'], 'bad'), (['Depressed'], 'bad'), (['Bad'], 'bad'), (['Lonely'], 'bad'), (['I', "'m", 'Okay'], 'neutral'), (['Fine'], 'neutral'), (['The', 'Usual'], 'neutral'), (['Okayish'], 'neutral'), (['Neutral'], 'neutral'), (['busy'], 'openended'), (['work'], 'openended'), (['life'], 'openended'), (['stress'], 'openended'), (['bored'], 'openended'), (['Angry'], 'openended'), (['Yes'], 'yes'), (['Okay'], 'yes'), (['Sure'], 'yes'), (['Yeah'], 'yes'), (['Yep'], 'yes'), (['No'], 'no'), (['Not', 'Really'], 'no'), (['Nope'], 'no')]
7 classes ['bad', 'good', 'greeting', 'neutral', 'no', 'openended', 'yes']
38 unique lemmatized words ["'m", 'angry', 'anyone', 'are', 'bad', 'bored', 'busy', 'depressed', 'fine', 'good', 'great', 'hello', 'h

In [4]:
pickle.dump(words,open('words.pkl','wb'))
pickle.dump(classes,open('classes.pkl','wb'))

training = []
output_empty = [0] * len(classes)

for doc in documents:
    bag = []
    pattern_words = doc[0]
    pattern_words = [lemmatizer.lemmatize(word.lower()) for word in pattern_words]
    for w in words:
        bag.append(1) if w in pattern_words else bag.append(0)

    output_row = list(output_empty)
    output_row[classes.index(doc[1])] = 1
    training.append([bag, output_row])
#     print(bag)
#     print(output_row)
#     print('\n\n')



random.shuffle(training)

training = np.array(training)
train_x = list(training[:,0])
train_y = list(training[:,1])
print("Training data created")

model = Sequential()
model.add(Dense(128, input_shape=(len(train_x[0]),), activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(train_y[0]), activation='softmax'))

sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])

hist = model.fit(np.array(train_x), np.array(train_y), epochs=200, batch_size=5, verbose=1)
model.save('chatbot_model.h5', hist)

print("model created")

Training data created
Epoch 1/200
31/31 [==============================] - 0s 6ms/step - loss: 1.9362 - accuracy: 0.2581
Epoch 2/200
31/31 [==============================] - 0s 611us/step - loss: 1.8946 - accuracy: 0.1613
Epoch 3/200
31/31 [==============================] - 0s 483us/step - loss: 1.8994 - accuracy: 0.3548
Epoch 4/200
31/31 [==============================] - 0s 483us/step - loss: 1.8386 - accuracy: 0.2903
Epoch 5/200
31/31 [==============================] - 0s 484us/step - loss: 1.8532 - accuracy: 0.2581
Epoch 6/200
31/31 [==============================] - 0s 418us/step - loss: 1.8152 - accuracy: 0.2581
Epoch 7/200
31/31 [==============================] - 0s 323us/step - loss: 1.8005 - accuracy: 0.3548
Epoch 8/200
31/31 [==============================] - 0s 354us/step - loss: 1.6647 - accuracy: 0.5161
Epoch 9/200
31/31 [==============================] - 0s 665us/step - loss: 1.6860 - accuracy: 0.4516
Epoch 10/200
31/31 [==============================] - 0s 685us/step - l

31/31 [==============================] - 0s 644us/step - loss: 0.1446 - accuracy: 0.9677
Epoch 82/200
31/31 [==============================] - ETA: 0s - loss: 0.2059 - accuracy: 1.00 - 0s 514us/step - loss: 0.1330 - accuracy: 0.9677
Epoch 83/200
31/31 [==============================] - 0s 452us/step - loss: 0.0811 - accuracy: 1.0000
Epoch 84/200
31/31 [==============================] - 0s 515us/step - loss: 0.1270 - accuracy: 0.9677
Epoch 85/200
31/31 [==============================] - 0s 600us/step - loss: 0.0472 - accuracy: 1.0000
Epoch 86/200
31/31 [==============================] - 0s 604us/step - loss: 0.0768 - accuracy: 1.0000
Epoch 87/200
31/31 [==============================] - 0s 613us/step - loss: 0.0387 - accuracy: 1.0000
Epoch 88/200
31/31 [==============================] - 0s 575us/step - loss: 0.0641 - accuracy: 1.0000
Epoch 89/200
31/31 [==============================] - 0s 613us/step - loss: 0.0947 - accuracy: 1.0000
Epoch 90/200
31/31 [==============================] -

31/31 [==============================] - 0s 580us/step - loss: 0.0548 - accuracy: 1.0000
Epoch 161/200
31/31 [==============================] - 0s 612us/step - loss: 0.0278 - accuracy: 1.0000
Epoch 162/200
31/31 [==============================] - 0s 610us/step - loss: 0.0228 - accuracy: 1.0000
Epoch 163/200
31/31 [==============================] - 0s 611us/step - loss: 0.0249 - accuracy: 1.0000
Epoch 164/200
31/31 [==============================] - 0s 611us/step - loss: 0.0353 - accuracy: 1.0000
Epoch 165/200
31/31 [==============================] - 0s 546us/step - loss: 0.0133 - accuracy: 1.0000
Epoch 166/200
31/31 [==============================] - 0s 644us/step - loss: 0.0696 - accuracy: 0.9677
Epoch 167/200
31/31 [==============================] - 0s 642us/step - loss: 0.0296 - accuracy: 1.0000
Epoch 168/200
31/31 [==============================] - 0s 598us/step - loss: 0.0262 - accuracy: 1.0000
Epoch 169/200
31/31 [==============================] - 0s 528us/step - loss: 0.0664 - a